In [0]:
#  Copyright 2019 Kiril Cvetkov. All Rights Reserved.


import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from keras import backend as K


In [0]:
class LSTMModule(tf.keras.Model):
  def __init__(self, n_filters):
    super(LSTMModule, self).__init__(name='')

    self.lstm1 = LSTM(n_filters)

    
    
  def call(self, inputs, training = False):
    x = self.lstm1(inputs)
    return x


data = tf.convert_to_tensor([0.1,0.2,0.3])

data = K.reshape(data,(1,3,1))

model = LSTMModule(1)
kiko = np.array([0.1,0.2,0.3])
kiko = kiko.reshape(1,3,1)
model.predict(kiko)




array([[-0.06181209]], dtype=float32)

In [0]:
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(1)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)
# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

[[0.07243214]]


In [0]:
inputs1 = Input(shape=(3, 1))
lstm1_output, state_h, state_c = LSTM(10, return_state=True)(inputs1)
#return state e okay ako vrzuvam edna so druga lstm
model = Model(inputs=inputs1, outputs=[lstm1_output, state_h, state_c])
# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
a,b, c = model.predict(data)

print(a==b)
print(a.shape)
print(b.shape)
print(c.shape)


[[ True  True  True  True  True  True  True  True  True  True]]
(1, 10)
(1, 10)
(1, 10)


In [0]:
inputs1 = Input(shape=(3, 1))
lstm1_output, state_h, state_c = LSTM(10, return_state=True, return_sequences=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1_output, state_h, state_c])
# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
a,b, c = model.predict(data)

print(a==b)
print(a.shape)
print(b.shape)
print(c.shape)

[[[False False False False False False False False False False]
  [False False False False False False False False False False]
  [ True  True  True  True  True  True  True  True  True  True]]]
(1, 3, 10)
(1, 10)
(1, 10)


In [0]:
inputs1 = Input(shape=(3, 1))
lstm1_output, state_h= GRU(10, return_state=True, return_sequences=True)(inputs1)
#return state e okay ako vrzuvam edna so druga lstm
model = Model(inputs=inputs1, outputs=[lstm1_output, state_h])
# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
a,b = model.predict(data)

print(a==b)
print(a.shape)
print(b.shape)



[[[False False False False False False False False False False]
  [False False False False False False False False False False]
  [ True  True  True  True  True  True  True  True  True  True]]]
(1, 3, 10)
(1, 10)


In [0]:
def get_models(n_input, n_output, n_units):
 	# define training encoder
  #n_input dict_size/hotencoding size
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model
  
model, encoder_model, decoder_model = get_models(1,1, 10)


In [0]:
def predict_sentence(inference_encoder, inference_decoder, source, n_steps, cardinality):
  #encode
  state = infenc.predict(source)
  
  target_seq = np.array([0]*cardinality).reshape(1,1,cardinality)
  
  output = list()
  
  for t in range(n_steps):
    yhat, h, c = infdec.predict([target_seq]+state)
    output.append(yhat[0,0,:])
    state = [h,c,]
    target_sequence = yhat
    
  return np.array(output)
  #
  

In [0]:
from random import *
from keras.utils import to_categorical

# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]

def get_dataset(n_in, n_out, cardinality, n_samples):
  X1, X2, y = list(), list(), list()
  for _ in range(n_samples):
    # generate source sequence
    source = generate_sequence(n_in, cardinality)
    # define target sequence
    target = source[:n_out]
    target.reverse()
    # create padded input target sequence
    target_in = [0] + target[:-1]
    # encode
#     print(source)
#     print(target)
#     print(target_in)
    src_encoded = to_categorical([source], num_classes=cardinality)
    tar_encoded = to_categorical([target], num_classes=cardinality)
    tar2_encoded = to_categorical([target_in], num_classes=cardinality)
    # store
    X1.append(src_encoded)
    X2.append(tar2_encoded)
    y.append(tar_encoded)
  return np.array(X1), np.array(X2), np.array(y)

In [0]:
get_dataset(6,3, 60, 1)

(array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [0]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
  # encode
  state = infenc.predict(source)
  # start of sequence input
  target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
  # collect predictions
  output = list()
  for t in range(n_steps):
    # predict next char
    yhat, h, c = infdec.predict([target_seq] + state)
    # store prediction
    output.append(yhat[0,0,:])
    print(yhat.shape)
    print("prediction:",one_hot_decode(yhat[0]))
    # update state
    state = [h, c]
    # update target sequence

    target_seq = yhat
  return np.array(output)

In [0]:
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3

In [0]:

# define model
train, infenc, infdec = get_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
X1 = X1.squeeze()
X2 = X2.squeeze()
y = y.squeeze()
print(X1.shape,X2.shape,y.shape)
# train model
train.fit([X1, X2], y, epochs=1)

(100000, 6, 51) (100000, 3, 51) (100000, 3, 51)
Epoch 1/1
100000/100000 [==============================] - 93s 932us/step - loss: 0.6396 - acc: 0.7954


In [0]:
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]

total, correct = 100, 0
for _ in range(total):
  X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)

  X1=X1.reshape(X1.shape[1:])
  X2=X2.reshape(X2.shape[1:])
  y = y.reshape(y.shape[1:])

  target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
  if np.array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
    correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))


prediction: [5]
prediction: [7]
prediction: [34]
prediction: [42]
prediction: [8]
prediction: [12]
prediction: [13]
prediction: [21]
prediction: [21]
prediction: [34]
prediction: [12]
prediction: [41]
prediction: [4]
prediction: [47]
prediction: [14]
prediction: [27]
prediction: [42]
prediction: [4]
prediction: [2]
prediction: [30]
prediction: [20]
prediction: [38]
prediction: [45]
prediction: [34]
prediction: [13]
prediction: [8]
prediction: [47]
prediction: [49]
prediction: [43]
prediction: [10]
prediction: [10]
prediction: [6]
prediction: [33]
prediction: [32]
prediction: [42]
prediction: [15]
prediction: [29]
prediction: [19]
prediction: [26]
prediction: [44]
prediction: [8]
prediction: [26]
prediction: [15]
prediction: [1]
prediction: [5]
prediction: [34]
prediction: [17]
prediction: [48]
prediction: [17]
prediction: [25]
prediction: [11]
prediction: [16]
prediction: [22]
prediction: [2]
prediction: [19]
prediction: [45]
prediction: [35]
prediction: [19]
prediction: [41]
predictio

In [0]:
# spot check some examples
for _ in range(10):
  X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
  X1=X1.reshape(X1.shape[1:])
  y = y.reshape(y.shape[1:])

  target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
  print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

(1, 1, 51)
prediction: [1]
(1, 1, 51)
prediction: [20]
(1, 1, 51)
prediction: [9]
X=[9, 20, 1, 13, 13, 10] y=[1, 20, 9], yhat=[1, 20, 9]
(1, 1, 51)
prediction: [21]
(1, 1, 51)
prediction: [5]
(1, 1, 51)
prediction: [14]
X=[14, 5, 21, 31, 48, 32] y=[21, 5, 14], yhat=[21, 5, 14]
(1, 1, 51)
prediction: [9]
(1, 1, 51)
prediction: [38]
(1, 1, 51)
prediction: [49]
X=[49, 38, 9, 22, 47, 21] y=[9, 38, 49], yhat=[9, 38, 49]
(1, 1, 51)
prediction: [24]
(1, 1, 51)
prediction: [35]
(1, 1, 51)
prediction: [6]
X=[6, 35, 24, 31, 43, 27] y=[24, 35, 6], yhat=[24, 35, 6]
(1, 1, 51)
prediction: [3]
(1, 1, 51)
prediction: [31]
(1, 1, 51)
prediction: [10]
X=[10, 31, 3, 49, 5, 12] y=[3, 31, 10], yhat=[3, 31, 10]
(1, 1, 51)
prediction: [11]
(1, 1, 51)
prediction: [10]
(1, 1, 51)
prediction: [33]
X=[33, 10, 11, 15, 25, 29] y=[11, 10, 33], yhat=[11, 10, 33]
(1, 1, 51)
prediction: [4]
(1, 1, 51)
prediction: [20]
(1, 1, 51)
prediction: [49]
X=[49, 20, 4, 32, 50, 16] y=[4, 20, 49], yhat=[4, 20, 49]
(1, 1, 51)
pre

In [0]:
#so se desava vo generiranje recenici, ima samo encoder :) pak moze istoto da se simulira so for 1Cell, pa drug cell

#ZA UTRE probaj so 1d konvolucija bez dekoder

In [0]:
20, 5, 120 => 